In [ ]:
#%pip install wget
#%pip install torch
import data_rnn
import numpy as np
import torch

In [3]:
(x_train, y_train), (x_val, y_val), (i2w, w2i), numcls = data_rnn.load_imdb(final=False)

# Part 1: Classification: data loading

In [4]:
x_train[0], y_train[0] # 0 is positive, 1 is negative

([14, 19, 9, 379, 22, 11, 50, 52, 53, 290], 1)

In [5]:
[i2w[word] for word in x_train[0]]
len(i2w)

99430

In [6]:
lengths = [len(l) for l in x_train]
print(np.min(lengths), np.max(lengths), np.average(lengths))

10 2514 240.6318


In [7]:
w2i['.pad'], w2i['.start'], w2i['.end'], w2i['.unk']

(0, 1, 2, 3)

## Question 1: Padding and Conversion

In [8]:
def pad(seq, pad_length):
    padded = np.zeros(pad_length) # 0 is for padding
    padded[0:len(seq)] = seq
    return torch.tensor(padded, dtype=torch.long)

In [9]:
padded = pad(x_train[0], 12).reshape(1,-1)
padded.shape

torch.Size([1, 12])

# Part 2: Classification, baseline model

## Question 2: Baseline model

In [10]:
# Take a tensor x, and return max across time dimension
def MaxPoolTime(x):
    return torch.amax(x, dim=1)

class q2_baseline(torch.nn.Module):
    def __init__(self, batch_size=1):
        super().__init__()
        timestep = 12
        numcls = 2
        hidden = 300
        embedding_size = 300
        n_embeddings = len(i2w)
        self.emb = torch.nn.Embedding(n_embeddings, embedding_size, padding_idx=0)
        self.fc1 = torch.nn.Linear(embedding_size, hidden)
        self.fc2 = torch.nn.Linear(hidden, numcls)
    
    def forward(self, x):
        x = self.emb(x)
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        x = MaxPoolTime(x)
        x = self.fc2(x)

        return x

In [11]:
baseline_model = q2_baseline()
y = baseline_model.forward(padded)
torch.nn.functional.softmax(y)

/var/folders/zw/n1g26qmn72ldwxh52bkgncv40000gn/T/ipykernel_17660/3313671974.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.nn.functional.softmax(y)


tensor([[0.5154, 0.4846]], grad_fn=<SoftmaxBackward0>)

## Question 3: Training Loop

In [12]:
# pad all
padding_size = np.max(lengths)

padded_train = torch.stack([pad(x, padding_size) for x in x_train])
padded_val = torch.stack([pad(x, padding_size) for x in x_val])

In [13]:
padded_train.shape

torch.Size([20000, 2514])

Create train and validation datasets

In [22]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 32

# create train and val datasets with instance and label pairs
train_dataset = TensorDataset(padded_train, torch.tensor(y_train))
validation_dataset = TensorDataset(padded_val, torch.tensor(y_val))

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)


In [15]:
import torch.optim as optim
# setup training loop

optimizer = optim.Adam(baseline_model.parameters(), lr=0.001)

In [16]:
def validate(model, validation_loader):
    val_acc = 0
    val_correct = 0
    total_samples = 0
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(validation_loader):
            instances, labels = data
            fwd = model(instances)
            predictions = torch.argmax(fwd, dim=1)
            val_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)

    # Compute and return accuracy
    val_acc = val_correct / total_samples
    return val_acc

Training loop

In [26]:
baseline_model = q2_baseline(batch_size)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, batch in enumerate(trainloader, 0):
        instances, labels = batch

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = baseline_model(instances)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0
    val_acc = validate(baseline_model, testloader)
    print(f'Epoch {epoch}, validation acc.: {val_acc}')
print('Finished Training')

[1,   200] loss: 0.713
[1,   400] loss: 0.708
[1,   600] loss: 0.713
Epoch 0, validation acc.: 0.4926
[2,   200] loss: 0.714


# Part 3: Writing your own Elman RNN

To be completed!